In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cardiovascular_Disease"
cohort = "GSE276839"

# Input paths
in_trait_dir = "../../input/GEO/Cardiovascular_Disease"
in_cohort_dir = "../../input/GEO/Cardiovascular_Disease/GSE276839"

# Output paths
out_data_file = "../../output/preprocess/Cardiovascular_Disease/GSE276839.csv"
out_gene_data_file = "../../output/preprocess/Cardiovascular_Disease/gene_data/GSE276839.csv"
out_clinical_data_file = "../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE276839.csv"
json_path = "../../output/preprocess/Cardiovascular_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Connecting Transcriptomics with Computational Modeling to Reveal Developmental Adaptations in Pediatric Human Atrial Tissue"
!Series_summary	"Nearly 1% of babies are born with congenital heart disease (CHD) – many of whom will require heart surgery within the first few years of life. A detailed understanding of cardiac maturation can help to expand our knowledge on cardiac diseases that develop during gestation, identify age-appropriate drug therapies, and inform clinical care decisions related to surgical repair and postoperative management. Yet, to date, our knowledge of the temporal changes that cardiomyocytes undergo during postnatal development is limited. In this study, we collected right atrial tissue samples from pediatric patients (n=117) undergoing heart surgery. Patients were stratified into five age groups. We measured age-dependent adaptations in cardiac gene expression, and used computational modeling to simulate action potential and

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os

# Load clinical data
clinical_data = pd.DataFrame({0: ['age group: Adolescent/Young Adult', 'age group: Infant', 'age group: School Age', 'age group: Toddler/Pre School', 'age group: Neonate', 'age group: infant']})

# 1. Gene Expression Data Availability
# Based on the background information, this study measures age-dependent adaptations in cardiac gene expression
# So gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 For trait (Cardiovascular Disease)
# From the background info, all patients have congenital heart disease (CHD)
# This is a constant feature as all patients have the same condition
trait_row = None  # Trait data not usable for association studies since all patients have CHD

# 2.2 For age
# Age information is available in row 0 as age groups
age_row = 0

def convert_age(value):
    if not value or ':' not in value:
        return None
    
    age_str = value.split(':', 1)[1].strip().lower()
    
    # Convert age groups to numerical values (approximating median age in each group)
    if 'neonate' in age_str:
        return 0  # 0-1 month
    elif 'infant' in age_str:
        return 0.5  # ~6 months
    elif 'toddler' in age_str or 'pre school' in age_str:
        return 3  # ~3 years
    elif 'school age' in age_str:
        return 9  # ~9 years
    elif 'adolescent' in age_str or 'young adult' in age_str:
        return 16  # ~16 years
    else:
        return None

# 2.3 For gender
# Gender information is not available in the provided data
gender_row = None

def convert_gender(value):
    # Not used but defined for completeness
    if not value or ':' not in value:
        return None
    
    gender_str = value.split(':', 1)[1].strip().lower()
    
    if 'female' in gender_str or 'f' == gender_str:
        return 0
    elif 'male' in gender_str or 'm' == gender_str:
        return 1
    else:
        return None

# Define convert_trait function (though not used)
def convert_trait(value):
    # All patients have CHD, so would return 1 if used
    return 1

# 3. Save Metadata - initial filtering
# Check if trait data is available (it's not in this case)
is_trait_available = trait_row is not None

# Save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Cardiovascular_Disease/GSE276839/GSE276839_series_matrix.txt.gz
Gene data shape: (2792, 117)
First 20 gene/probe identifiers:
Index(['TC0100006587.hg.1', 'TC0100006681.hg.1', 'TC0100006723.hg.1',
       'TC0100006725.hg.1', 'TC0100006781.hg.1', 'TC0100006849.hg.1',
       'TC0100006875.hg.1', 'TC0100006912.hg.1', 'TC0100006937.hg.1',
       'TC0100007194.hg.1', 'TC0100007207.hg.1', 'TC0100007295.hg.1',
       'TC0100007334.hg.1', 'TC0100007458.hg.1', 'TC0100007463.hg.1',
       'TC0100007529.hg.1', 'TC0100007574.hg.1', 'TC0100007645.hg.1',
       'TC0100007841.hg.1', 'TC0100007846.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, they appear to be probe IDs from a custom or commercial microarray/transcript cluster platform
# with the format "TCxxxxxxxx.hg.1" rather than standard HGNC gene symbols like "BRCA1" or "TP53"
# These are likely Affymetrix or Thermo Fisher transcript cluster IDs that need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'category', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine the gene annotation data to identify appropriate columns for mapping
# From the annotation data, we can see:
# - 'ID' column contains the probe identifiers (e.g., TC0100006437.hg.1) that match gene_data
# - 'SPOT_ID.1' column contains long descriptions that include gene symbols

# 2. Create a mapping dataframe using the 'ID' column and extract gene symbols from 'SPOT_ID.1'
mapping_df = gene_annotation[['ID', 'SPOT_ID.1']]
mapping_df = mapping_df.rename(columns={'SPOT_ID.1': 'Gene'})

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
# The function extract_human_gene_symbols will parse the gene symbols from the annotations
# The apply_gene_mapping function handles the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Display information about the gene expression data
print(f"\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols:")
print(gene_data.index[:5])

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")



Gene expression data after mapping:
Shape: (12539, 117)
First 5 gene symbols:
Index(['A0A087WTH5', 'A0A087X093', 'A0A096LNQ9', 'A0A096LNT9', 'A0A0B4J1W7'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Cardiovascular_Disease/gene_data/GSE276839.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols
print(f"Original gene data shape: {gene_data.shape}")

try:
    # Attempt to normalize gene symbols
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Normalized gene data shape: {gene_data_normalized.shape}")
except Exception as e:
    print(f"Gene normalization failed: {e}")
    # If normalization fails, use the original gene data
    gene_data_normalized = gene_data.copy()
    print(f"Using original gene data with shape: {gene_data_normalized.shape}")

# Save the gene expression data 
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Re-examine the clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics:")
print(sample_characteristics_dict)

# Define proper age group conversion function
def convert_age_group(value):
    if not isinstance(value, str) or ':' not in value:
        return None
    
    age_str = value.split(':', 1)[1].strip().lower()
    
    # Convert age groups to numerical values
    if 'neonate' in age_str:
        return 0
    elif 'infant' in age_str:
        return 1
    elif 'toddler' in age_str or 'pre school' in age_str:
        return 2
    elif 'school age' in age_str:
        return 3
    elif 'adolescent' in age_str or 'young adult' in age_str:
        return 4
    else:
        return None

# Extract clinical features using age group as the trait
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data, 
    trait="Age_Group",  # Using age group as trait
    trait_row=0,
    convert_trait=convert_age_group,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

print(f"Clinical features shape: {clinical_features.shape}")
print("Clinical features preview:")
print(preview_df(clinical_features))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data_normalized)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# Handle missing values
linked_data_clean = handle_missing_values(linked_data, "Age_Group")
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# Check for bias in the dataset
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, "Age_Group")

# Conduct final quality validation
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note="Dataset contains gene expression data from patients in different age groups (neonate, infant, toddler, school-age, adolescent/young adult). The study examines age-dependent adaptations in cardiac gene expression."
)

# Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Original gene data shape: (12539, 117)
Normalized gene data shape: (3014, 117)


Gene expression data saved to ../../output/preprocess/Cardiovascular_Disease/gene_data/GSE276839.csv
Sample characteristics:
{0: ['age group: Adolescent/Young Adult', 'age group: Infant', 'age group: School Age', 'age group: Toddler/Pre School', 'age group: Neonate', 'age group: infant']}
Clinical features shape: (1, 117)
Clinical features preview:
{'GSM8506502': [4.0], 'GSM8506503': [1.0], 'GSM8506504': [1.0], 'GSM8506505': [3.0], 'GSM8506506': [1.0], 'GSM8506507': [2.0], 'GSM8506508': [0.0], 'GSM8506509': [2.0], 'GSM8506510': [3.0], 'GSM8506511': [2.0], 'GSM8506512': [4.0], 'GSM8506513': [4.0], 'GSM8506514': [2.0], 'GSM8506515': [4.0], 'GSM8506516': [2.0], 'GSM8506517': [2.0], 'GSM8506518': [1.0], 'GSM8506519': [1.0], 'GSM8506520': [1.0], 'GSM8506521': [1.0], 'GSM8506522': [1.0], 'GSM8506523': [1.0], 'GSM8506524': [4.0], 'GSM8506525': [2.0], 'GSM8506526': [2.0], 'GSM8506527': [4.0], 'GSM8506528': [4.0], 'GSM8506529': [1.0], 'GSM8506530': [3.0], 'GSM8506531': [1.0], 'GSM8506532': [3.0

Linked data shape after handling missing values: (117, 3015)
Quartiles for 'Age_Group':
  25%: 1.0
  50% (Median): 2.0
  75%: 3.0
Min: 0.0
Max: 4.0
The distribution of the feature 'Age_Group' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cardiovascular_Disease/GSE276839.csv
